In [68]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, \
    Flatten, Lambda, LSTM, RepeatVector, TimeDistributed, Reshape, \
    Conv2D, MaxPooling2D, BatchNormalization, ConvLSTM2D
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.utils.np_utils import to_categorical

In [3]:
# # models
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.resnet50 import ResNet50

In [3]:
data = h5py.File("../dataset/data_sample.hdf5", "r")
print("data keys: " + str(list(data.keys())))

data keys: ['images', 'labels', 'observation_days']


In [4]:
images = data["images"][:]
print("image shape: (sample, x_size, y_size, epoch) = " + str(images.shape))
labels = data["labels"][:]

image shape: (sample, x_size, y_size, epoch) = (72000, 21, 21, 48)


In [53]:
def txt2digit(labels):
    dic = {'Asteroids':0, 'Constant':1, 'EmptyLigh':2, 'M33Cephei':3, 'RRLyrae':4, 'Supernova':5}
    labels_digit = np.array([dic[i] for i in labels])
    return labels_digit

In [52]:
def build_dataset(images, labels, seq2seq=False):
    
    train_indices = np.random.choice(np.arange(images.shape[0]), int( 0.7 * images.shape[0]))
    val_indices = list(set(np.arange(images.shape[0])) - set(train_indices))
    
    x = np.transpose(images, (0, 3, 1, 2))
    x = np.expand_dims(x, len(x.shape))
    y = to_categorical(txt2digit(labels))
    
    if seq2seq:
        y = np.repeat(y, 48).reshape(-1, 48, 6)
    
    return x, y

In [54]:
x, y = build_dataset(images, labels)

In [ ]:
## shuffle and row selection are
# very slow
# x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3)
## maybe flatten -> select -> reshape back ??

In [65]:
def init_model(cnn_input_dim=21, cnn_output_dim=128, cnn_dropout=0.5,
               rnn_hidden_dim=128, rnn_output_dim=64, num_classes=6, rnn_dropout=0.5, timestep=48):
    
    intput_shape = (timestep, cnn_input_dim, cnn_input_dim, 1)
    
    model = Sequential()
    
    # CNN
    model.add(TimeDistributed(Conv2D(32, (4,4), \
                     padding='same', activation='relu', kernel_initializer='uniform'), input_shape=intput_shape))
    model.add(BatchNormalization())
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(Dropout(cnn_dropout))
    model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(BatchNormalization())
    model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(BatchNormalization())
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(BatchNormalization())
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(BatchNormalization())
    model.add(TimeDistributed(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(Dropout(cnn_dropout))
    model.add(BatchNormalization())
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    model.add(TimeDistributed(Flatten()))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(BatchNormalization())
    model.add(Dropout(cnn_dropout))
    model.add(TimeDistributed(Dense(cnn_output_dim, activation='relu')))
    
    # RNN
    # rnn_input_dim = cnn_output_dim
    # encode
    model.add(LSTM(rnn_hidden_dim, input_shape=(timestep, cnn_output_dim), dropout=rnn_dropout))
    # repeat vector for timestep
    #model.add(RepeatVector(timestep))
    # decode
    # TODO: do some research on stacked LSTM
    #model.add(LSTM(rnn_hidden_dim, dropout=rnn_dropout, return_sequences=True))
    # add dense
    # model.add(TimeDistributed(Dense(num_classes, activation='softmax')))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [61]:
model = init_model()

In [66]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_40 (TimeDis (None, 48, 21, 21, 32)    544       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 21, 21, 32)    128       
_________________________________________________________________
time_distributed_41 (TimeDis (None, 48, 19, 19, 32)    0         
_________________________________________________________________
time_distributed_42 (TimeDis (None, 48, 19, 19, 64)    18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 19, 19, 64)    256       
_________________________________________________________________
time_distributed_43 (TimeDis (None, 48, 19, 19, 64)    36928     
_________________________________________________________________
batch_normalization_3 (Batch (None, 48, 19, 19, 64)    256       
__________

In [ ]:
def init_model2():
    
    model = Sequential()
    
    model.add(ConvLSTM2D(32, (3, 3), input_shape=(48, 21, 21, 1), data_format='channels_last'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(6, activation='softmax'))
    
    
    

In [63]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# batch_size = 32
# epochs = 20

# model.fit(x[:1000], y[:1000],
#               batch_size=batch_size,
#               epochs=epochs,
#               validation_split=0.2)

In [235]:
def train(trainset, testset, cnn, rnn, batch_size=128, num_epoch=10, lr=0.0001, 
          print_every=100, early_stop=True):
    pass

In [99]:
model = Sequential()
model.add(ConvLSTM2D(32, (3, 3), input_shape=(48, 21, 21, 1), data_format='channels_last'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [102]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [103]:
model.fit(x[:1000], y[:1000],
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,
              shuffle=True)

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 151s 189ms/step - loss: 2.3262 - acc: 0.1963 - val_loss: 1.7322 - val_acc: 0.2950
Epoch 2/20
800/800 [==============================] - 148s 185ms/step - loss: 1.6664 - acc: 0.3387 - val_loss: 1.7673 - val_acc: 0.2100
Epoch 3/20
800/800 [==============================] - 145s 182ms/step - loss: 1.5841 - acc: 0.3238 - val_loss: 1.5600 - val_acc: 0.2850
Epoch 4/20
800/800 [==============================] - 146s 182ms/step - loss: 1.4140 - acc: 0.3737 - val_loss: 1.4169 - val_acc: 0.3200
Epoch 5/20
800/800 [==============================] - 142s 178ms/step - loss: 1.3245 - acc: 0.3950 - val_loss: 1.3929 - val_acc: 0.3600
Epoch 6/20
800/800 [==============================] - 147s 184ms/step - loss: 1.2545 - acc: 0.4175 - val_loss: 1.2597 - val_acc: 0.4400
Epoch 7/20
800/800 [==============================] - 154s 193ms/step - loss: 1.1556 - acc: 0.4838 - val_loss: 1.2823 - val_acc: 0.3750
Ep

KeyboardInterrupt: 